In [1]:
%%bash

./env.sh

  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6497 sha256=d9a5f48ddbf6bf3171da65503cc5aa564ac1b85b39597720687c074aa9e22978
  Stored in directory: /root/.cache/pip/wheels/fe/4e/02/c9c3154e4845bfdbf1fdf344f5a89f16dcbb4f627a908c9974
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-py3-none-any.whl size=7532 sha256=f3984d7c9fd9a89f1bd39b22ed71956d9827a6c198b95d600c853e0a78e2006c
  Stored in directory: /root/.cache/pip/wheels/3b/75/17/b11b16ad90276ff6e4e03ec375d55291a186c7fe9dbf87fba3
  Created wheel for visdom: filename=visdom-0.2.3-py3-none-any.whl size=1417080 sha256=bf95406511da5586ec2b8ece1523b0f3f914c80b6374c4d36219e0fc4ba86b8d
  Stored in directory: /root/.cache/pip/wheels/4d/73/32/0bbe55d2dccb9d80d3f020f474c15a5a1eef232817dcebe776
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=615c8a0744ab183bf45a15eba8b2add3e0566f95d03f2d1ba52c80985ce4179b
  Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc9

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import json
import os
import random
import time
import pdb

import torch
import torch.backends.cudnn as cudnn

In [3]:
class CFG:
    model_method = 'fcn'
    phase = 'train'

In [4]:
from lib.models.nets.hrnet import HRNet_W48_Proto
import argparse
parser = argparse.ArgumentParser()

DATA_ROOT = '/tmp/working/workspace/ProtoSeg_local/data'
SCRATCH_ROOT = '/tmp/working/workspace/ProtoSeg_local/output'
ASSET_ROOT = '/tmp/working/workspace/ProtoSeg_local/checkpoints/cityscapes'

BACKBONE="hrnet48"
DATA_DIR = DATA_ROOT + '/Cityscapes'
SAVE_DIR = SCRATCH_ROOT + '/Cityscapes/seg_results/'
# SAVE_DIR = '/tmp/working/workspace/PP_seg/output/Cityscapes/seg_results/'

CHECKPOINTS_ROOT = SCRATCH_ROOT + "/Cityscapes"

MODEL_NAME="hrnet_w48_proto"
LOSS_TYPE="pixel_prototype_ce_loss"
CHECKPOINTS_NAME = "hrnet_w48_proto_lr1x_hrnet_proto_80k"

PRETRAINED_MODEL = ASSET_ROOT + "/hrnetv2_w48_imagenet_pretrained.pth"
MAX_ITERS=80000
BATCH_SIZE=8
BASE_LR=0.01

In [5]:
def main(): 
    def str2bool(v):
        """ Usage:
        parser.add_argument('--pretrained', type=str2bool, nargs='?', const=True,
                            dest='pretrained', help='Whether to use pretrained models.')
        """
        if v.lower() in ('yes', 'true', 't', 'y', '1'):
            return True
        elif v.lower() in ('no', 'false', 'f', 'n', '0'):
            return False
        else:
            raise argparse.ArgumentTypeError('Unsupported value encountered.')
            
    # train & test config
    parser.add_argument('--phase', default='train', type=str, dest='phase')
    parser.add_argument('--model_name', default="hrnet_w48_proto", type=str, dest='model_name')
    parser.add_argument('--configs', default='configs/cityscapes/H_48_D_4_proto.json', type=str,dest='configs', help='The file of the hyper parameters.')
    parser.add_argument('--gpu', default=[0], nargs='+', type=int, dest='gpu', help='The gpu list used.')
    parser.add_argument('--data_dir', default=None, type=str, nargs='+', dest='data:data_dir', help='The Directory of the data.')
    parser.add_argument('--backbone', default=None, type=str, dest='network:backbone', help='The base network of model.')
    parser.add_argument('--checkpoints_name', default='hrnet_w48_proto_lr1x_hrnet_proto_80k', type=str, dest='checkpoints:checkpoints_name', help='The name of checkpoint model.')
    parser.add_argument('--loss_type', default='pixel_prototype_ce_loss', type=str, dest='loss:loss_type', help='The loss type of the network.')
    parser.add_argument('--drop_last', type=str2bool, nargs='?', default=False, dest='data:drop_last', help='Fix bug for syncbn.')
    
    # train config
    parser.add_argument('--gathered', type=str2bool, nargs='?', default=True, dest='network:gathered', help='Whether to gather the output of model.')
    parser.add_argument('--loss_balance', type=str2bool, nargs='?', default=False, dest='network:loss_balance', help='Whether to balance GPU usage.')
    parser.add_argument('--log_to_file', type=str2bool, nargs='?', default=True, dest='logging:log_to_file', help='Whether to write logging into files.')
    parser.add_argument('--max_iters', default=None, type=int, dest='solver:max_iters', help='The max iters of training.')
    parser.add_argument('--checkpoints_root', default=None, type=str, dest='checkpoints:checkpoints_root', help='The root dir of model save path.')
    parser.add_argument('--pretrained', type=str, default=None, dest='network:pretrained', help='The path to pretrained model.')
    parser.add_argument('--train_batch_size', default=None, type=int, dest='train:batch_size', help='The batch size of training.')
    parser.add_argument('--distributed', action='store_true', dest='distributed', help='Use multi-processing training.')
    parser.add_argument('--base_lr', default=None, type=float, dest='lr:base_lr', help='The learning rate.')
    parser.add_argument('--nbb_mult', default=1.0, type=float, dest='lr:nbb_mult', help='The not backbone mult ratio of learning rate.')

    # test config
    parser.add_argument('--network', default='hrnet_w48_proto', type=str, dest='network')
    parser.add_argument('REMAIN', nargs='*')
    parser.add_argument('--test_img', default=None, type=str, dest='test:test_img', help='The test path of image.')
    parser.add_argument('--test_dir', default=None, type=str, dest='test:test_dir', help='The test directory of images.')
    parser.add_argument('--out_dir', default='none', type=str, dest='test:out_dir', help='The test out directory of images.')
    parser.add_argument('--save_prob', type=str2bool, nargs='?', default=False, dest='test:save_prob', help='Save the logits map during testing.')
    parser.add_argument('--resume', default=None, type=str, dest='network:resume', help='The path of checkpoints.')

    print('pass parser set')
    
    # python
    #args = parser.parse_args()

    # jupyter
    # args_parser = parser.parse_args(args=['--phase',
    #                                       '--model_method',
    #                                       '--network',
    #                                       '--model_name'])

    args_parser = parser.parse_args(args=['--drop_last','y',
                                         '--gathered','n',
                                         '--loss_balance','y',
                                         '--log_to_file','n',
                                         '--data_dir',DATA_DIR,
                                         '--backbone', BACKBONE,
                                         '--model_name',MODEL_NAME,
                                         '--loss_type ',LOSS_TYPE,
                                         '--max_iters', '80000',
                                         '--checkpoints_root', CHECKPOINTS_ROOT,
                                         '--pretrained',PRETRAINED_MODEL,
                                         '--train_batch_size', '8',
                                         '--distributed',
                                         '--base_lr', '0.01',])
    
    return args_parser


# Utils

In [6]:
import argparse
import json
import os
import sys

from ast import literal_eval

class Configer(object):

    def __init__(self, args_parser=None, configs=None, config_dict=None):
        if config_dict is not None:
            self.params_root = config_dict

        elif configs is not None:
            if not os.path.exists(configs):
                exit(0)

            json_stream = open(configs, 'r')
            self.params_root = json.load(json_stream)
            json_stream.close()

        elif args_parser is not None:
            self.args_dict = args_parser.__dict__
            self.params_root = None

            if not os.path.exists(args_parser.configs):
                print('Json Path:{} not exists!'.format(args_parser.configs))
                exit(1)

            json_stream = open(args_parser.configs, 'r')
            self.params_root = json.load(json_stream)
            json_stream.close()

            for key, value in self.args_dict.items():
                if not self.exists(*key.split(':')):
                    self.add(key.split(':'), value)
                elif value is not None:
                    self.update(key.split(':'), value)

            self._handle_remaining_args(args_parser.REMAIN)

        self.conditions = _ConditionHelper(self)


    def _handle_remaining_args(self, remain):

        def _parse_value(x: str):
            """
            We first try to parse `x` as python literal object.
            If failed, we regard x as string.
            """
            try:
                return literal_eval(x)
            except ValueError:
                return x

        def _set_value(key, value):
            """
            We directly operate on `params_root`.
            """
            remained_parts = key.split('.')
            consumed_parts = []

            parent_dict = self.params_root
            while len(remained_parts) > 1:
                cur_key = remained_parts.pop(0)
                consumed_parts.append(cur_key)

                if cur_key not in parent_dict:
                    parent_dict[cur_key] = dict()
                elif not isinstance(parent_dict[cur_key], dict):
                    sys.exit(1)
                
                parent_dict = parent_dict[cur_key]

            cur_key = remained_parts.pop(0)
            consumed_parts.append(cur_key)

            if cur_key.endswith('+'):
                cur_key = cur_key[:-1]
                target = parent_dict.get(cur_key)

                if not isinstance(target, list):
                    sys.exit(1)

                target.append(value)
                return

            existing_value = parent_dict.get(cur_key)
#             if existing_value is not None:
#             else:
            parent_dict[cur_key] = value

        assert len(remain) % 2 == 0, remain
        args = {}
        for i in range(len(remain) // 2):
            key, value = remain[2 * i: 2 * i + 2]
            _set_value(key, _parse_value(value))

    def clone(self):
        from copy import deepcopy
        return Configer(config_dict=deepcopy(self.params_root))

    def _get_caller(self):
        filename = os.path.basename(sys._getframe().f_back.f_back.f_code.co_filename)
        lineno = sys._getframe().f_back.f_back.f_lineno
        prefix = '{}, {}'.format(filename, lineno)
        return prefix

    def get(self, *key):
        if len(key) == 0:
            return self.params_root

        elif len(key) == 1:
            if key[0] in self.params_root:
                return self.params_root[key[0]]
            else:
                exit(1)

        elif len(key) == 2:
            if key[0] in self.params_root and key[1] in self.params_root[key[0]]:
                return self.params_root[key[0]][key[1]]
            else:
                exit(1)

        else:
            exit(1)

    def exists(self, *key):
        if len(key) == 1 and key[0] in self.params_root:
            return True

        if len(key) == 2 and (key[0] in self.params_root and key[1] in self.params_root[key[0]]):
            return True

        return False

    def add(self, key_tuple, value):
        if self.exists(*key_tuple):
            exit(1)

        if len(key_tuple) == 1:
            self.params_root[key_tuple[0]] = value

        elif len(key_tuple) == 2:
            if key_tuple[0] not in self.params_root:
                self.params_root[key_tuple[0]] = dict()

            self.params_root[key_tuple[0]][key_tuple[1]] = value

        else:
            exit(1)

    def update(self, key_tuple, value):
        if not self.exists(*key_tuple):
            exit(1)

        if len(key_tuple) == 1 and not isinstance(self.params_root[key_tuple[0]], dict):
            self.params_root[key_tuple[0]] = value

        elif len(key_tuple) == 2:
            self.params_root[key_tuple[0]][key_tuple[1]] = value

        else:
            exit(1)

    def resume(self, config_dict):
        self.params_root = config_dict

    def plus_one(self, *key):
        if not self.exists(*key):
            exit(1)

        if len(key) == 1 and not isinstance(self.params_root[key[0]], dict):
            self.params_root[key[0]] += 1

        elif len(key) == 2:
            self.params_root[key[0]][key[1]] += 1

        else:
            exit(1)

    def to_dict(self):
        return self.params_root


class _ConditionHelper:
    """Handy helper"""

    def __init__(self, configer):
        self.configer = configer

    @property
    def use_multi_dataset(self):
        root_dirs = self.configer.get('data', 'data_dir')
        return isinstance(root_dirs, (tuple, list)) and len(root_dirs) > 1

    @property
    def pred_sw_offset(self):
        return self.configer.exists('data', 'pred_sw_offset')

    @property
    def pred_dt_offset(self):
        return self.configer.exists('data', 'pred_dt_offset')

    @property
    def use_sw_offset(self):
        return self.configer.exists('data', 'use_sw_offset')

    @property
    def use_dt_offset(self):
        return self.configer.exists('data', 'use_dt_offset')

    @property
    def use_ground_truth(self):
        return self.config_equals(('use_ground_truth',), True)

    @property
    def pred_ml_dt_offset(self):
        return self.configer.exists('data', 'pred_ml_dt_offset')

    def loss_contains(self, name):
        return name in self.configer.get('loss', 'loss_type')

    def model_contains(self, name):
        return name in self.configer.get('network', 'model_name')

    def config_equals(self, key, value):
        if not self.configer.exists(*key):
            return False

        return self.configer.get(*key) == value

    def config_exists(self, key):
        return self.configer.exists(*key)

    def environ_exists(self, key):
        return os.environ.get(key) is not None

    @property
    def diverse_size(self):
        return self.configer.get('val', 'data_transformer')['size_mode'] == 'diverse_size'

class AverageMeter(object):
    """ Computes ans stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


# DataLoader

In [7]:
import pdb
import torch
from torch.utils import data

import lib.datasets.tools.transforms as trans
import lib.datasets.tools.cv2_aug_transforms as cv2_aug_trans
import lib.datasets.tools.pil_aug_transforms as pil_aug_trans
from lib.datasets.loader.default_loader import DefaultLoader, CSDataTestLoader
from lib.datasets.tools.collate import collate
from lib.utils.tools.logger import Logger as Log

from lib.utils.distributed import get_world_size, get_rank, is_distributed

class DataLoader(object):

    def __init__(self, configer):
        self.configer = configer

        from lib.datasets.tools import cv2_aug_transforms
        self.aug_train_transform = cv2_aug_transforms.CV2AugCompose(self.configer, split='train')
        self.aug_val_transform = cv2_aug_transforms.CV2AugCompose(self.configer, split='val')

        self.img_transform = trans.Compose([
            trans.ToTensor(),
            trans.Normalize(div_value=self.configer.get('normalize', 'div_value'),
                            mean=self.configer.get('normalize', 'mean'),
                            std=self.configer.get('normalize', 'std')), ])

        self.label_transform = trans.Compose([
            trans.ToLabel(),
            trans.ReLabel(255, -1), ])
 
    def get_dataloader_sampler(self, klass, split, dataset):

        from lib.datasets.loader.multi_dataset_loader import MultiDatasetLoader, MultiDatasetTrainingSampler

        root_dir = self.configer.get('data', 'data_dir')
        if isinstance(root_dir, list) and len(root_dir) == 1:
            root_dir = root_dir[0]

        kwargs = dict(
            dataset=dataset,
            aug_transform=(self.aug_train_transform if split == 'train' else self.aug_val_transform),
            img_transform=self.img_transform,
            label_transform=self.label_transform,
            configer=self.configer
        )

        if isinstance(root_dir, str):
            loader = klass(root_dir, **kwargs)
            multi_dataset = False
        elif isinstance(root_dir, list):
            loader = MultiDatasetLoader(root_dir, klass, **kwargs)
            multi_dataset = True
            Log.info('use multi-dataset for {}...'.format(dataset))
        else:
            raise RuntimeError('Unknown root dir {}'.format(root_dir))

        if split == 'train':
            if is_distributed() and multi_dataset:
                raise RuntimeError('Currently multi dataset doesn\'t support distributed.')

            if is_distributed():
                sampler = torch.utils.data.distributed.DistributedSampler(loader)
            elif multi_dataset:
                sampler = MultiDatasetTrainingSampler(loader)
            else:
                sampler = None

        elif split == 'val':

            if is_distributed():
                sampler = torch.utils.data.distributed.DistributedSampler(loader)
            else:
                sampler = None

        return loader, sampler

    def get_trainloader(self):
        if self.configer.exists('data', 'use_edge') and self.configer.get('data', 'use_edge') == 'ce2p':
            """
            ce2p manner:
            load both the ground-truth label and edge.
            """
            Log.info('use edge (follow ce2p) for train...')
            klass = LipLoader

        elif self.configer.exists('data', 'use_dt_offset') or self.configer.exists('data', 'pred_dt_offset'):
            """
            dt-offset manner:
            load both the ground-truth label and offset (based on distance transform).
            """
            Log.info('use distance transform offset loader for train...')
            klass = DTOffsetLoader

        elif self.configer.exists('train', 'loader') and \
                (self.configer.get('train', 'loader') == 'ade20k'
                 or self.configer.get('train', 'loader') == 'pascal_context'
                 or self.configer.get('train', 'loader') == 'pascal_voc'
                 or self.configer.get('train', 'loader') == 'coco_stuff'
                 or self.configer.get('train', 'loader') == 'camvid'):
            """
            ADE20KLoader manner:
            support input images of different shapes.
            """
            Log.info('use ADE20KLoader (diverse input shape) for train...')
            klass = ADE20KLoader
        else:
            """
            Default manner:
            + support input images of the same shapes.
            + support distributed training (the performance is more un-stable than non-distributed manner)
            """
            Log.info('use the DefaultLoader for train...')
            klass = DefaultLoader
        loader, sampler = self.get_dataloader_sampler(klass, 'train', 'train')
        trainloader = data.DataLoader(
            loader,
            batch_size=self.configer.get('train', 'batch_size') // get_world_size(), pin_memory=True,
            num_workers=self.configer.get('data', 'workers') // get_world_size(),
            sampler=sampler,
            shuffle=(sampler is None),
            drop_last=self.configer.get('data', 'drop_last'),
            collate_fn=lambda *args: collate(
                *args, trans_dict=self.configer.get('train', 'data_transformer')
            )
        )
        return trainloader
    
    def get_valloader(self, dataset=None):
        dataset = 'val' if dataset is None else dataset

        if self.configer.exists('data', 'use_dt_offset') or self.configer.exists('data', 'pred_dt_offset'):
            """
            dt-offset manner:
            load both the ground-truth label and offset (based on distance transform).
            """
            Log.info('use distance transform based offset loader for val ...')
            klass = DTOffsetLoader

        elif self.configer.get('method') == 'fcn_segmentor':
            """
            default manner:
            load the ground-truth label.
            """
            Log.info('use DefaultLoader for val ...')
            klass = DefaultLoader
        else:
            Log.error('Method: {} loader is invalid.'.format(self.configer.get('method')))
            return None

        loader, sampler = self.get_dataloader_sampler(klass, 'val', dataset)
        valloader = data.DataLoader(
            loader,
            sampler=sampler,
            batch_size=self.configer.get('val', 'batch_size') // get_world_size(), pin_memory=False,
            num_workers=self.configer.get('data', 'workers'), shuffle=False,
            collate_fn=lambda *args: collate(
                *args, trans_dict=self.configer.get('val', 'data_transformer')
            )
        )
        return valloader

    def get_testloader(self, dataset=None):
        dataset = 'test' if dataset is None else dataset
        if self.configer.exists('data', 'use_sw_offset') or self.configer.exists('data', 'pred_sw_offset'):
            Log.info('use sliding window based offset loader for test ...')
            test_loader = data.DataLoader(
                SWOffsetTestLoader(root_dir=self.configer.get('data', 'data_dir'), dataset=dataset,
                                   img_transform=self.img_transform,
                                   configer=self.configer),
                batch_size=self.configer.get('test', 'batch_size'), pin_memory=False,
                num_workers=self.configer.get('data', 'workers'), shuffle=False,
                collate_fn=lambda *args: collate(
                    *args, trans_dict=self.configer.get('test', 'data_transformer')
                )
            )
            return test_loader

        elif self.configer.get('method') == 'fcn_segmentor':
            Log.info('use CSDataTestLoader for test ...')

            root_dir = self.configer.get('data', 'data_dir')
            if isinstance(root_dir, list) and len(root_dir) == 1:
                root_dir = root_dir[0]
            test_loader = data.DataLoader(
                CSDataTestLoader(root_dir=root_dir, dataset=dataset,
                                 img_transform=self.img_transform,
                                 configer=self.configer),
                batch_size=self.configer.get('test', 'batch_size'), pin_memory=True,
                num_workers=self.configer.get('data', 'workers'), shuffle=False,
                collate_fn=lambda *args: collate(
                    *args, trans_dict=self.configer.get('test', 'data_transformer')
                )
            )
            return test_loader

In [8]:
from lib.models.nets.hrnet import HRNet_W48_Proto

SEG_MODEL_DICT = {
    'hrnet_w48_proto': HRNet_W48_Proto
}

class Segmentation_Model(object):
    def __init__(self, configer):
        self.configer = configer

    def semantic_segmentor(self):
        # ここよくわからん
        #model_name = self.configer.get('network', 'model_name')
        model_name = 'hrnet_w48_proto'

        if model_name not in SEG_MODEL_DICT:
            print("your select model not found")
        print(model_name)
        model = SEG_MODEL_DICT[model_name](self.configer)

        return model

# Loss

In [9]:
from lib.loss.loss_proto import PixelPrototypeCELoss

SEG_LOSS_DICT = {
    'pixel_prototype_ce_loss': PixelPrototypeCELoss
}

class LossManager(object):
    def __init__(self, configer):
        self.configer = configer

    def _parallel(self, loss):
        if is_distributed():
            print('use distributed loss')
            return loss

    def get_seg_loss(self, loss_type=None):
        key = self.configer.get('loss', 'loss_type') if loss_type is None else loss_type
        if key not in SEG_LOSS_DICT:
            Log.error('Loss: {} not valid!'.format(key))
            exit(1)
        Log.info('use loss: {}.'.format(key))
        loss = SEG_LOSS_DICT[key](self.configer)
        return self._parallel(loss)

# optim & scheduler

In [10]:
import math

import torchcontrib
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from torch.optim.lr_scheduler import LambdaLR

from lib.utils.tools.logger import Logger as Log


class WarmupCosineSchedule(LambdaLR):
    """ Linear warmup and then cosine decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps following a cosine curve.
        If `cycles` (default=0.5) is different from default, learning rate follows cosine function after warmup.
    """

    def __init__(self, optimizer, warmup_steps, t_total, cycles=.5, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        return max(0.0, 0.5 * (1. + math.cos(math.pi * float(self.cycles) * 2.0 * progress)))


class OptimScheduler(object):
    def __init__(self, configer):
        self.configer = configer

    def init_optimizer(self, net_params):
        optimizer = None
        if self.configer.get('optim', 'optim_method') == 'sgd':
            optimizer = SGD(net_params,
                            lr=self.configer.get('lr', 'base_lr'),
                            momentum=self.configer.get('optim', 'sgd')['momentum'],
                            weight_decay=self.configer.get('optim', 'sgd')['weight_decay'],
                            nesterov=self.configer.get('optim', 'sgd')['nesterov'])

        elif self.configer.get('optim', 'optim_method') == 'adam':
            optimizer = Adam(net_params,
                             lr=self.configer.get('lr', 'base_lr'),
                             betas=self.configer.get('optim', 'adam')['betas'],
                             eps=self.configer.get('optim', 'adam')['eps'],
                             weight_decay=self.configer.get('optim', 'adam')['weight_decay'])
        elif self.configer.get('optim', 'optim_method') == 'adamw':
            optimizer = AdamW(net_params,
                              lr=self.configer.get('lr', 'base_lr'),
                              betas=self.configer.get('optim', 'adamw')['betas'],
                              eps=self.configer.get('optim', 'adamw')['eps'],
                              weight_decay=self.configer.get('optim', 'adamw')['weight_decay'])

        else:
            Log.error('Optimizer {} is not valid.'.format(self.configer.get('optim', 'optim_method')))
            exit(1)

        policy = self.configer.get('lr', 'lr_policy')

        scheduler = None
        if policy == 'step':
            scheduler = lr_scheduler.StepLR(optimizer,
                                            self.configer.get('lr', 'step')['step_size'],
                                            gamma=self.configer.get('lr', 'step')['gamma'])

        elif policy == 'multistep':
            scheduler = lr_scheduler.MultiStepLR(optimizer,
                                                 self.configer.get('lr', 'multistep')['stepvalue'],
                                                 gamma=self.configer.get('lr', 'multistep')['gamma'])

        elif policy == 'lambda_poly':
            if os.environ.get('lambda_poly_power'):
                _lambda_poly_power = float(os.environ.get('lambda_poly_power'))
                Log.info('Use lambda_poly policy with power {}'.format(_lambda_poly_power))
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')),
                                                _lambda_poly_power)
            elif self.configer.exists('lr', 'lambda_poly'):
                Log.info('Use lambda_poly policy with power {}'.format(self.configer.get('lr', 'lambda_poly')['power']))
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')),
                                                self.configer.get('lr', 'lambda_poly')['power'])
            else:
                Log.info('Use lambda_poly policy with default power 0.9')
                lambda_poly = lambda iters: pow((1.0 - iters / self.configer.get('solver', 'max_iters')), 0.9)
            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_poly)

        elif policy == 'lambda_cosine':
            lambda_cosine = lambda iters: (math.cos(math.pi * iters / self.configer.get('solver', 'max_iters'))
                                           + 1.0) / 2
            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_cosine)

        elif policy == 'plateau':
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode=self.configer.get('lr', 'plateau')['mode'],
                                                       factor=self.configer.get('lr', 'plateau')['factor'],
                                                       patience=self.configer.get('lr', 'plateau')['patience'],
                                                       threshold=self.configer.get('lr', 'plateau')['threshold'],
                                                       threshold_mode=self.configer.get('lr', 'plateau')['thre_mode'],
                                                       cooldown=self.configer.get('lr', 'plateau')['cooldown'],
                                                       min_lr=self.configer.get('lr', 'plateau')['min_lr'],
                                                       eps=self.configer.get('lr', 'plateau')['eps'])

        elif policy == 'swa_lambda_poly':
            optimizer = torchcontrib.optim.SWA(optimizer)
            normal_max_iters = int(self.configer.get('solver', 'max_iters') * 0.75)
            swa_step_max_iters = (self.configer.get('solver',
                                                    'max_iters') - normal_max_iters) // 5 + 1  # we use 5 ensembles here

            def swa_lambda_poly(iters):
                if iters < normal_max_iters:
                    return pow(1.0 - iters / normal_max_iters, 0.9)
                else:  # set lr to half of initial lr and start swa
                    return 0.5 * pow(1.0 - ((iters - normal_max_iters) % swa_step_max_iters) / swa_step_max_iters, 0.9)

            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=swa_lambda_poly)

        elif policy == 'swa_lambda_cosine':
            optimizer = torchcontrib.optim.SWA(optimizer)
            normal_max_iters = int(self.configer.get('solver', 'max_iters') * 0.75)
            swa_step_max_iters = (self.configer.get('solver',
                                                    'max_iters') - normal_max_iters) // 5 + 1  # we use 5 ensembles here

            def swa_lambda_cosine(iters):
                if iters < normal_max_iters:
                    return (math.cos(math.pi * iters / normal_max_iters) + 1.0) / 2
                else:  # set lr to half of initial lr and start swa
                    return 0.5 * (math.cos(
                        math.pi * ((iters - normal_max_iters) % swa_step_max_iters) / swa_step_max_iters) + 1.0) / 2

            scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=swa_lambda_cosine)

        elif policy == 'warmup_cosine':
            scheduler = WarmupCosineSchedule(optimizer, warmup_steps=1000,
                                             t_total=self.configer.get('solver', 'max_iters'))

        else:
            Log.error('Policy:{} is not valid.'.format(policy))
            exit(1)

        return optimizer, scheduler

    def update_optimizer(self, net, optim_method, lr_policy):
        self.configer.update(('optim', 'optim_method'), optim_method)
        self.configer.update(('lr', 'lr_policy'), lr_policy)
        optimizer, scheduler = self.init_optimizer(net)
        return optimizer, scheduler

# Train

In [11]:
import time

import os
import cv2
import pdb
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import torch.backends.cudnn as cudnn

# from lib.models.model_manager import ModelManager
from lib.utils.tools.logger import Logger as Log
from lib.vis.seg_visualizer import SegVisualizer
from segmentor.tools.module_runner import ModuleRunner
from segmentor.tools.data_helper import DataHelper
from segmentor.tools.evaluator import get_evaluator
from lib.utils.distributed import get_world_size, get_rank, is_distributed

2022-11-06 15:41:21,959 INFO    [offset_helper.py, 51] engery/max-distance: 5 engery/min-distance: 0
2022-11-06 15:41:21,959 INFO    [offset_helper.py, 58] direction/num_classes: 8 scale: 1
2022-11-06 15:41:21,960 INFO    [offset_helper.py, 65] c4 align axis: False


In [12]:
class Trainer(object):
    def __init__(self, configer):
        self.configer = configer
        self.batch_time = AverageMeter()
        self.foward_time = AverageMeter()
        self.backward_time = AverageMeter()
        self.loss_time = AverageMeter()
        self.data_time = AverageMeter()
        self.train_losses = AverageMeter()
        # self.val_losses = AverageMeter()
        self.seg_visualizer = SegVisualizer(configer)
        self.loss_manager = LossManager(configer)
        self.module_runner = ModuleRunner(configer)
        self.model_manager = Segmentation_Model(configer)
        self.data_loader = DataLoader(configer)
        self.optim_scheduler = OptimScheduler(configer)
        self.data_helper = DataHelper(configer, self)
        self.evaluator = get_evaluator(configer, self)

        self.seg_net = None
        self.train_loader = None
        self.val_loader = None
        self.optimizer = None
        self.scheduler = None
        self.running_score = None

        self._init_model()

    def _init_model(self):
        self.seg_net = self.model_manager.semantic_segmentor()

        try:
            flops, params = get_model_complexity_info(self.seg_net, (3, 512, 512))
            split_line = '=' * 30
            print('{0}\nInput shape: {1}\nFlops: {2}\nParams: {3}\n{0}'.format(
                split_line, (3, 512, 512), flops, params))
            print('!!!Please be cautious if you use the results in papers. '
                  'You may need to check if all ops are supported and verify that the '
                  'flops computation is correct.')
        except:
            pass

        self.seg_net = self.module_runner.load_net(self.seg_net)

        if self.configer.get('optim', 'group_method') == 'decay':
            params_group = self.group_weight(self.seg_net)
        else:
            assert self.configer.get('optim', 'group_method') is None
            params_group = self._get_parameters()

        self.optimizer, self.scheduler = self.optim_scheduler.init_optimizer(params_group)

        self.train_loader = self.data_loader.get_trainloader()
        self.val_loader = self.data_loader.get_valloader()
        self.pixel_loss = self.loss_manager.get_seg_loss()
        if is_distributed():
            self.pixel_loss = self.module_runner.to_device(self.pixel_loss)

        self.with_proto = True if self.configer.exists("protoseg") else False


    @staticmethod
    def group_weight(module):
        group_decay = []
        group_no_decay = []
        for m in module.modules():
            if isinstance(m, nn.Linear):
                group_decay.append(m.weight)
                if m.bias is not None:
                    group_no_decay.append(m.bias)
            elif isinstance(m, nn.modules.conv._ConvNd):
                group_decay.append(m.weight)
                if m.bias is not None:
                    group_no_decay.append(m.bias)
            else:
                if hasattr(m, 'weight'):
                    group_no_decay.append(m.weight)
                if hasattr(m, 'bias'):
                    group_no_decay.append(m.bias)

        assert len(list(module.parameters())) == len(group_decay) + len(group_no_decay)
        groups = [dict(params=group_decay), dict(params=group_no_decay, weight_decay=.0)]
        return groups
    
    def _get_parameters(self):
        bb_lr = []
        nbb_lr = []
        fcn_lr = []
        params_dict = dict(self.seg_net.named_parameters())
        for key, value in params_dict.items():
            if 'backbone' in key:
                bb_lr.append(value)
            elif 'aux_layer' in key or 'upsample_proj' in key:
                fcn_lr.append(value)
            else:
                nbb_lr.append(value)
        params = [{'params': bb_lr, 'lr': self.configer.get('lr', 'base_lr')},
                  {'params': fcn_lr, 'lr': self.configer.get('lr', 'base_lr') * 10},
                  {'params': nbb_lr, 'lr': self.configer.get('lr', 'base_lr') * self.configer.get('lr', 'nbb_mult')}]
        return params
    
    def __train(self):
        """
          Train function of every epoch during train phase.
        """
        self.seg_net.train()
        self.pixel_loss.train()
        start_time = time.time()
        scaler = torch.cuda.amp.GradScaler()

        if "swa" in self.configer.get('lr', 'lr_policy'):
            normal_max_iters = int(self.configer.get('solver', 'max_iters') * 0.75)
            swa_step_max_iters = (self.configer.get('solver', 'max_iters') - normal_max_iters) // 5 + 1

        if hasattr(self.train_loader.sampler, 'set_epoch'):
            self.train_loader.sampler.set_epoch(self.configer.get('epoch'))

        for i, data_dict in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            if self.configer.get('lr', 'metric') == 'iters':
                self.scheduler.step(self.configer.get('iters'))
            else:
                self.scheduler.step(self.configer.get('epoch'))

            if self.configer.get('lr', 'is_warm'):
                self.module_runner.warm_lr(
                    self.configer.get('iters'),
                    self.scheduler, self.optimizer, backbone_list=[0, ]
                )

            (inputs, targets), batch_size = self.data_helper.prepare_data(data_dict)
            self.data_time.update(time.time() - start_time)

            foward_start_time = time.time()
            with torch.cuda.amp.autocast():
                if not self.with_proto:
                    outputs = self.seg_net(*inputs)
                else:
                    pretrain_prototype = True if self.configer.get('iters') < self. configer.get('protoseg', 'warmup_iters') else False
                    outputs = self.seg_net(*inputs, gt_semantic_seg=targets[:, None, ...],
                                           pretrain_prototype=pretrain_prototype)
            self.foward_time.update(time.time() - foward_start_time)

            loss_start_time = time.time()
            if is_distributed():
                import torch.distributed as dist
                def reduce_tensor(inp):
                    """
                    Reduce the loss from all processes so that 
                    process with rank 0 has the averaged results.
                    """
                    world_size = get_world_size()
                    if world_size < 2:
                        return inp
                    with torch.no_grad():
                        reduced_inp = inp
                        dist.reduce(reduced_inp, dst=0)
                    return reduced_inp

                with torch.cuda.amp.autocast():
                    loss = self.pixel_loss(outputs, targets)
                    backward_loss = loss
                    display_loss = reduce_tensor(backward_loss) / get_world_size()
            else:
                backward_loss = display_loss = self.pixel_loss(outputs, targets)

            self.train_losses.update(display_loss.item(), batch_size)
            self.loss_time.update(time.time() - loss_start_time)

            backward_start_time = time.time()

            # backward_loss.backward()
            # self.optimizer.step()
            scaler.scale(backward_loss).backward()
            scaler.step(self.optimizer)
            scaler.update()

            self.backward_time.update(time.time() - backward_start_time)

            # Update the vars of the train phase.
            self.batch_time.update(time.time() - start_time)
            start_time = time.time()
            self.configer.plus_one('iters')

            # Print the log info & reset the states.
            if self.configer.get('iters') % self.configer.get('solver', 'display_iter') == 0 and \
                    (not is_distributed() or get_rank() == 0):
                
                self.batch_time.reset()
                self.foward_time.reset()
                self.backward_time.reset()
                self.loss_time.reset()
                self.data_time.reset()
                self.train_losses.reset()

            # save checkpoints for swa
            if 'swa' in self.configer.get('lr', 'lr_policy') and \
                    self.configer.get('iters') > normal_max_iters and \
                    ((self.configer.get('iters') - normal_max_iters) % swa_step_max_iters == 0 or \
                     self.configer.get('iters') == self.configer.get('solver', 'max_iters')):
                self.optimizer.update_swa()

            if self.configer.get('iters') == self.configer.get('solver', 'max_iters'):
                break

            # Check to val the current model.
            # if self.configer.get('epoch') % self.configer.get('solver', 'test_interval') == 0:
            if self.configer.get('iters') % self.configer.get('solver', 'test_interval') == 0:
                self.__val()

        self.configer.plus_one('epoch')

    def __val(self, data_loader=None):
        """
          Validation function during the train phase.
        """
        self.seg_net.eval()
        self.pixel_loss.eval()
        start_time = time.time()
        replicas = self.evaluator.prepare_validaton()

        data_loader = self.val_loader if data_loader is None else data_loader
        for j, data_dict in enumerate(data_loader):
            if j % 10 == 0:
                if is_distributed(): dist.barrier()  # Synchronize all processes

            if self.configer.get('dataset') == 'lip':
                (inputs, targets, inputs_rev, targets_rev), batch_size = self.data_helper.prepare_data(data_dict,
                                                                                                       want_reverse=True)
            else:
                (inputs, targets), batch_size = self.data_helper.prepare_data(data_dict)

            with torch.no_grad():
                if self.configer.get('dataset') == 'lip':
                    inputs = torch.cat([inputs[0], inputs_rev[0]], dim=0)
                    outputs = self.seg_net(inputs)
                    if not is_distributed():
                        outputs_ = self.module_runner.gather(outputs)
                    else:
                        outputs_ = outputs
                    if isinstance(outputs_, (list, tuple)):
                        outputs_ = outputs_[-1]
                    outputs = outputs_[0:int(outputs_.size(0) / 2), :, :, :].clone()
                    outputs_rev = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), :, :, :].clone()
                    if outputs_rev.shape[1] == 20:
                        outputs_rev[:, 14, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 15, :, :]
                        outputs_rev[:, 15, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 14, :, :]
                        outputs_rev[:, 16, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 17, :, :]
                        outputs_rev[:, 17, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 16, :, :]
                        outputs_rev[:, 18, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 19, :, :]
                        outputs_rev[:, 19, :, :] = outputs_[int(outputs_.size(0) / 2):int(outputs_.size(0)), 18, :, :]
                    outputs_rev = torch.flip(outputs_rev, [3])
                    outputs = (outputs + outputs_rev) / 2.
                    self.evaluator.update_score(outputs, data_dict['meta'])

                elif self.data_helper.conditions.diverse_size:
                    if is_distributed():
                        outputs = [self.seg_net(inputs[i]) for i in range(len(inputs))]
                    else:
                        outputs = nn.parallel.parallel_apply(replicas[:len(inputs)], inputs)

                    for i in range(len(outputs)):
                        loss = self.pixel_loss(outputs[i], targets[i].unsqueeze(0))
                        # self.val_losses.update(loss.item(), 1)
                        outputs_i = outputs[i]
                        if isinstance(outputs_i, torch.Tensor):
                            outputs_i = [outputs_i]
                        self.evaluator.update_score(outputs_i, data_dict['meta'][i:i + 1])

                else:
                    outputs = self.seg_net(*inputs)

                    if not is_distributed():
                        outputs = self.module_runner.gather(outputs)
                    if isinstance(outputs, dict):
                        outputs = outputs['seg']
                    self.evaluator.update_score(outputs, data_dict['meta'])

            self.batch_time.update(time.time() - start_time)
            start_time = time.time()

        self.evaluator.update_performance()

        self.module_runner.save_net(self.seg_net, save_mode='performance')
        cudnn.benchmark = True

        # Print the log info & reset the states.
        self.evaluator.reduce_scores()
        if not is_distributed() or get_rank() == 0:
            self.evaluator.print_scores()

        self.batch_time.reset()
        self.evaluator.reset()
        self.seg_net.train()
        self.pixel_loss.train()

    def train(self):
        # cudnn.benchmark = True
        # self.__val()
        if self.configer.get('network', 'resume') is not None:
            if self.configer.get('network', 'resume_val'):
                self.__val(data_loader=self.data_loader.get_valloader(dataset='val'))
                return
            elif self.configer.get('network', 'resume_train'):
                print('train')
                self.__val(data_loader=self.data_loader.get_valloader(dataset='train'))
                return
            # return

        # if self.configer.get('network', 'resume') is not None and self.configer.get('network', 'resume_val'):
        #     self.__val(data_loader=self.data_loader.get_valloader(dataset='val'))
        #     return

        while self.configer.get('iters') < self.configer.get('solver', 'max_iters'):
            self.__train()

        # use swa to average the model
        if 'swa' in self.configer.get('lr', 'lr_policy'):
            self.optimizer.swap_swa_sgd()
            self.optimizer.bn_update(self.train_loader, self.seg_net)

        self.__val(data_loader=self.data_loader.get_valloader(dataset='val'))

    def summary(self):
        from lib.utils.summary import get_model_summary
        import torch.nn.functional as F
        self.seg_net.eval()

        for j, data_dict in enumerate(self.train_loader):
            print(get_model_summary(self.seg_net, data_dict['img'][0:1]))
            return


In [13]:
from lib.utils.helpers.file_helper import FileHelper
from lib.utils.helpers.image_helper import ImageHelper
from lib.utils.tools.logger import Logger as Log
from lib.metrics.running_score import RunningScore
from lib.vis.seg_visualizer import SegVisualizer
from lib.vis.palette import get_cityscapes_colors
from segmentor.tools.module_runner import ModuleRunner
from segmentor.tools.optim_scheduler import OptimScheduler

import fnmatch
import platform

import torch.distributed as dist
dist.init_process_group('gloo', init_method='file:///tmp/somefile', rank=0, world_size=1)

2022-11-06 15:41:22,012 INFO    Added key: store_based_barrier_key:1 to store for rank: 0


In [14]:
parser = argparse.ArgumentParser()

In [15]:
configer = Configer(args_parser = main())
data_dir = configer.get('data', 'data_dir')

pass parser set


In [16]:
if isinstance(data_dir, str):
    data_dir = [data_dir]
abs_data_dir = [os.path.expanduser(x) for x in data_dir]
configer.update(['data', 'data_dir'], abs_data_dir)
            
#             project_dir = os.path.dirname(os.path.realpath(__file__))
#             configer.add(['project_dir'], project_dir)
model = None

model =Trainer(configer)

2022-11-06 15:41:22,024 INFO    [module_runner.py, 46] BN Type is torchsyncbn.
2022-11-06 15:41:22,025 INFO    [__init__.py, 17] Using evaluator: StandardEvaluator
2022-11-06 15:41:22,025 INFO    [running_score.py, 129] 19


hrnet_w48_proto


2022-11-06 15:41:22,376 INFO    [module_helper.py, 141] Loading pretrained model:/tmp/working/workspace/ProtoSeg_local/checkpoints/cityscapes/hrnetv2_w48_imagenet_pretrained.pth
2022-11-06 15:41:22,482 INFO    [module_helper.py, 205] Missing keys: []
2022-11-06 15:41:23,899 INFO    [optim_scheduler.py, 96] Use lambda_poly policy with default power 0.9
2022-11-06 15:41:23,899 INFO    [<ipython-input-7-fb31557d580a>, 114] use the DefaultLoader for train...
2022-11-06 15:41:23,950 INFO    [default_loader.py, 38] train 2975
2022-11-06 15:41:23,950 INFO    [<ipython-input-7-fb31557d580a>, 146] use DefaultLoader for val ...
2022-11-06 15:41:23,954 INFO    [default_loader.py, 38] val 500
2022-11-06 15:41:23,955 INFO    [<ipython-input-9-70c2063975e7>, 21] use loss: pixel_prototype_ce_loss.
2022-11-06 15:41:23,955 INFO    [loss_proto.py, 60] ignore_index: -1


use distributed loss


In [17]:
from lib.utils.helpers.file_helper import FileHelper
from lib.utils.helpers.image_helper import ImageHelper
from lib.utils.tools.logger import Logger as Log
from lib.metrics.running_score import RunningScore
from lib.vis.seg_visualizer import SegVisualizer
from lib.vis.palette import get_cityscapes_colors
from segmentor.tools.module_runner import ModuleRunner
from segmentor.tools.optim_scheduler import OptimScheduler

import fnmatch
import platform

import torch.distributed as dist
dist.init_process_group('gloo', init_method='file:///tmp/somefile', rank=0, world_size=1)


if __name__ == "__main__":
    INFERENCE = True
    parser = argparse.ArgumentParser()
    if CFG.model_method == 'fcn':
        if CFG.phase == 'infer':
            print('Load Model Now ...')
            configer = Configer(args_parser = main())
            data_dir = configer.get('data', 'data_dir')
            
            if isinstance(data_dir, str):
                data_dir = [data_dir]
            abs_data_dir = [os.path.expanduser(x) for x in data_dir]
            configer.update(['data', 'data_dir'], abs_data_dir)
            
#             project_dir = os.path.dirname(os.path.realpath(__file__))
#             configer.add(['project_dir'], project_dir)
            model = None
            
            model =Tester(configer)
            print('Load Model Finish')
            
            if INFERENCE:
                print('Inference Starting ...')
                args = inference_fn()
                cityscapes_evaluator = CityscapesEvaluator()
                cityscapes_evaluator.evaluate(pred_dir=args.pred_dir, gt_dir=args.gt_dir)
  
        else:
            print('Load Model Now ...')
            configer = Configer(args_parser = main())
            data_dir = configer.get('data', 'data_dir')
            
            if isinstance(data_dir, str):
                data_dir = [data_dir]
            abs_data_dir = [os.path.expanduser(x) for x in data_dir]
            configer.update(['data', 'data_dir'], abs_data_dir)
            
#             project_dir = os.path.dirname(os.path.realpath(__file__))
#             configer.add(['project_dir'], project_dir)
            model = None
            
            model =Trainer(configer)
            print('create model pass')
            
            print('start training model')
            model.train()
    else:
        print('attention models will be available')

RuntimeError: trying to initialize the default process group twice!